In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
import time
import collections

In [2]:
cation_name= pd.read_pickle('cation_name.pkl')
anion_name= pd.read_pickle('anion_name.pkl')
label= pd.read_pickle('label.pkl')
cat_reduced = pd.read_pickle('cat_reduced.pkl')
an_reduced = pd.read_pickle('an_reduced.pkl')

In [3]:
print(cation_name.shape)
print(anion_name.shape)
print(label.shape)
print(cat_reduced.shape)
print(an_reduced.shape)


(1762,)
(1762,)
(1762,)
(1762, 1090)
(1762, 672)


In [4]:
cat_reduced

,Cation_Mass,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,97.138,0,1,8.318418,2.198691,4.110920,8.318418,1.188345,2.861501,2.549529,...,8.206584,47.382536,97.076025,6.067252,41,4,32.0,34.0,2.972222,1.583333
1,142.261,0,1,12.349859,2.306090,4.483692,12.349859,1.234986,3.201531,2.815267,...,8.849514,52.733735,142.159026,4.738634,124,9,46.0,50.0,3.812500,2.375000
2,139.218,0,1,12.426800,2.236324,4.251712,12.426800,1.242680,3.191982,2.830660,...,8.497195,51.903908,139.122975,5.564919,132,8,44.0,47.0,3.722222,2.416667
3,150.240,0,1,13.725227,2.205946,4.411892,13.725227,1.247748,3.278326,3.019439,...,8.696677,40.025593,150.127726,5.560286,174,11,48.0,51.0,3.972222,2.666667
4,111.164,0,1,9.910095,2.224503,4.194610,9.910095,1.238762,2.984531,2.668822,...,8.365672,49.065586,111.091675,5.846930,63,6,36.0,39.0,3.222222,1.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,294.506,0,1,26.144381,2.581330,4.676624,26.144381,1.244971,3.948657,3.259417,...,9.817003,74.740815,294.290375,5.162989,1017,27,96.0,111.0,8.916667,5.166667
1790,294.506,0,1,26.144381,2.581330,4.676624,26.144381,1.244971,3.948657,3.259417,...,9.817003,74.740815,294.290375,5.162989,1017,27,96.0,111.0,8.916667,5.166667
1791,200.344,0,3,15.933892,2.283998,4.567996,15.933892,1.138135,3.475518,3.203757,...,9.035630,44.221074,200.212124,5.005303,314,18,58.0,63.0,7.444444,3.500000
1792,186.318,0,3,14.355432,2.263821,4.527642,14.355432,1.104264,3.402027,3.108404,...,8.941807,42.829266,186.196474,5.032337,262,16,54.0,58.0,7.194444,3.166667


Rename the columns to distinguish cations from anions

In [5]:
def rename_columns(df,addition):
    new_columns = [df.columns[0]] + [addition + col for col in df.columns[1:]]
    return df.rename(columns=dict(zip(df.columns, new_columns)))

# Apply the function to your dataframe
cat_renamed = rename_columns(cat_reduced,'cat:')
an_renamed = rename_columns(an_reduced,'an:')


In [6]:
print(cat_renamed.shape)
print(an_renamed.shape)


(1762, 1090)
(1762, 672)


In [7]:
features = pd.concat((cat_renamed, an_renamed), axis=1)

In [8]:
features

,Cation_Mass,cat:nAcid,cat:nBase,cat:SpAbs_A,cat:SpMax_A,cat:SpDiam_A,cat:SpAD_A,cat:SpMAD_A,cat:LogEE_A,cat:VE1_A,...,an:SRW09,an:SRW10,an:TSRW10,an:MW,an:AMW,an:WPath,an:WPol,an:Zagreb1,an:Zagreb2,an:mZagreb2
0,97.138,0,1,8.318418,2.198691,4.110920,8.318418,1.188345,2.861501,2.549529,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875
1,142.261,0,1,12.349859,2.306090,4.483692,12.349859,1.234986,3.201531,2.815267,...,0.0,8.815370,37.856472,179.924229,19.991581,88,6,42.0,40.0,1.750
2,139.218,0,1,12.426800,2.236324,4.251712,12.426800,1.242680,3.191982,2.830660,...,0.0,7.625107,29.418928,195.810897,39.162179,16,0,20.0,16.0,1.000
3,150.240,0,1,13.725227,2.205946,4.411892,13.725227,1.247748,3.278326,3.019439,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875
4,111.164,0,1,9.910095,2.224503,4.194610,9.910095,1.238762,2.984531,2.668822,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,294.506,0,1,26.144381,2.581330,4.676624,26.144381,1.244971,3.948657,3.259417,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875
1790,294.506,0,1,26.144381,2.581330,4.676624,26.144381,1.244971,3.948657,3.259417,...,0.0,6.192362,25.583106,66.009771,13.201954,20,2,14.0,12.0,1.500
1791,200.344,0,3,15.933892,2.283998,4.567996,15.933892,1.138135,3.475518,3.203757,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875
1792,186.318,0,3,14.355432,2.263821,4.527642,14.355432,1.104264,3.402027,3.108404,...,0.0,9.921769,47.846479,279.917842,18.661189,340,24,78.0,88.0,2.875


In [9]:
pos_idx = np.where(label == 1)[0] 
features.values[pos_idx].shape

(14, 1762)

In [10]:
idx_pos_test=[4,6,11]
idx_pos_train=list(set(pos_idx)-set(idx_pos_test))

In [11]:
print("The number of unique cations in the whole positive examples is:",len(set(cation_name[pos_idx])))
print("The number of unique anions in the whole positive examples is:",len(set(anion_name[pos_idx])))


The number of unique cations in the whole positive examples is: 8
The number of unique anions in the whole positive examples is: 9


In [12]:
print("The number of unique cations in the training positive examples is:",len(set(cation_name[idx_pos_train])))
print("The number of unique anions in the training positive examples is:",len(set(anion_name[idx_pos_train])))

The number of unique cations in the training positive examples is: 7
The number of unique anions in the training positive examples is: 8


In [13]:
X_train_pos=features.iloc[idx_pos_train]

In [14]:
X_train_pos=features.iloc[idx_pos_train]
X_test_pos=features.iloc[idx_pos_test]
y_train_pos=label[idx_pos_train]
y_test_pos= label[idx_pos_test]

In [26]:
counter = collections.Counter(y_train_pos)
print(counter)
counter = collections.Counter(y_test_pos)
print(counter)

Counter({1: 11})
Counter({1: 3})


In [15]:
print(X_train_pos.shape)
print(y_train_pos.shape)
print(X_test_pos.shape)
print(y_test_pos.shape)

(11, 1762)
(11,)
(3, 1762)
(3,)


In [31]:
neg_idx=pos_idx[-1]+1
neg_examples=features.iloc[neg_idx:]

In [32]:
X_train_neg, X_test_neg, y_train_neg, y_test_neg = train_test_split(neg_examples, label[neg_idx:], test_size=0.2, random_state=42)

In [33]:
print(X_train_neg.shape)
print(y_train_neg.shape)
print(X_test_neg.shape)
print(y_test_neg.shape)

(1398, 1762)
(1398,)
(350, 1762)
(350,)


In [34]:
counter = collections.Counter(y_train_neg)
print(counter)
counter = collections.Counter(y_test_neg)
print(counter)

Counter({-1: 1398})
Counter({-1: 350})


In [35]:
# Merge the training features (X) and labels (y)
X_train = pd.concat((X_train_pos, X_train_neg), axis=0)
y_train = pd.concat((y_train_pos, y_train_neg), axis=0)

# Merge the testing features (X) and labels (y)
X_test = pd.concat((X_test_pos, X_test_neg), axis=0)
y_test = pd.concat((y_test_pos, y_test_neg), axis=0)


In [36]:
counter = collections.Counter(y_train)
print(counter)

Counter({-1: 1398, 1: 11})


In [37]:
counter = collections.Counter(y_test)
print(counter)

Counter({-1: 350, 1: 3})


In [38]:
# Shuffle the training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
# Shuffle the testing data
X_test, y_test = shuffle(X_test, y_test, random_state=42)


In [39]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1409, 1762)
(1409,)
(353, 1762)
(353,)


In [40]:
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')
X_test.to_pickle('X_test.pkl')
y_test.to_pickle('y_test.pkl')

# Dealing with imbalanced with SMOTE (Synthetic Minority Oversampling TEchnique)

### PIPELINE

In [66]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import time



In [36]:
# Create a pipeline that first applies SMOTE and then scales the data
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=345)),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# Define the parameter grid for the SVM classifier
# Note: Specify 'svm__' before each parameter name to indicate these parameters are for the SVM step in the pipeline
param_grid = {
    'smote__k_neighbors': [2,3,4],  # Example range; adjust based on your dataset characteristics
    'svm__C': [0.1, 1, 10, 20],
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__degree': [2, 3, 4]  # Relevant for 'poly' kernel
}

# Create a grid search object with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=10)

In [37]:
start = time.time()
# Fit the grid search object to the training data (NOT the SMOTE-resampled data)
grid_search.fit(X_train, y_train)  # Use your original training data here
end = time.time()

dt = end - start
print('delta t =', dt)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5; 1/108] START smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear
[CV 1/5; 1/108] END smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear;, score=0.982 total time=   0.3s
[CV 2/5; 1/108] START smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear
[CV 2/5; 1/108] END smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear;, score=0.979 total time=   0.3s
[CV 3/5; 1/108] START smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear
[CV 3/5; 1/108] END smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear;, score=0.982 total time=   0.3s
[CV 4/5; 1/108] START smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear
[CV 4/5; 1/108] END smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear;, score=0.986 total time=   0.3s
[CV 5/5; 1/108] START smote__k_neighbors=2, svm__C=0.1, svm__degree=2, svm__kernel=linear
[CV

In [38]:
# Print results
results = grid_search.cv_results_
for mean_score, std_score, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    print(f"Mean test score: {mean_score:.3f} (+/-{std_score:.3f}) for {params}")

Mean test score: 0.977 (+/-0.012) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 2, 'svm__kernel': 'linear'}
Mean test score: 0.960 (+/-0.011) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 2, 'svm__kernel': 'rbf'}
Mean test score: 0.770 (+/-0.031) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 2, 'svm__kernel': 'poly'}
Mean test score: 0.977 (+/-0.012) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 3, 'svm__kernel': 'linear'}
Mean test score: 0.960 (+/-0.011) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 3, 'svm__kernel': 'rbf'}
Mean test score: 0.577 (+/-0.031) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 3, 'svm__kernel': 'poly'}
Mean test score: 0.977 (+/-0.012) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 4, 'svm__kernel': 'linear'}
Mean test score: 0.960 (+/-0.011) for {'smote__k_neighbors': 2, 'svm__C': 0.1, 'svm__degree': 4, 'svm__kernel': 'rbf'}
Mean test score: 0.436 (+/-0.034) for

In [39]:
# Print the best parameters and score
print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)
# Access the best model
best_model = grid_search.best_estimator_

Best parameters:  {'smote__k_neighbors': 2, 'svm__C': 10, 'svm__degree': 2, 'svm__kernel': 'rbf'}
Best score:  0.9886524822695035


In [40]:
best_model

Pipeline(steps=[('smote', SMOTE(k_neighbors=2, random_state=345)),
                ('scaler', StandardScaler()), ('svm', SVC(C=10, degree=2))])

In [41]:
# Train the best model on the entire dataset
# best_model.fit(X_train,y_train)
#Actually, we don't need to manually refir the best model to the entire dataset. It has already been done automatically if refit=True in GridSearchCV which is the default

In [42]:
y_pred = grid_search.best_estimator_.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

Test Set Accuracy: 98.02%


In [43]:
# Print the classification report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       1.00      0.98      0.99       350
           1       0.30      1.00      0.46         3

    accuracy                           0.98       353
   macro avg       0.65      0.99      0.73       353
weighted avg       0.99      0.98      0.99       353



In [44]:
y_pred[np.where(y_test==1)]

array([1, 1, 1])

In [45]:
np.sum(y_pred==1)

10

In [46]:
np.sum(y_test==-1)

350

In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[343   7]
 [  0   3]]
